## 문제정의
* Given
  * 사전 학습 된 DNN 모델이 주어짐
  * 모델 구조는 아래 코드 참고
  * 사전 학습에 사용한 학습 데이터가 주어짐
  * 테스트 데이터가 주어짐
  * 사전 학습 한 모델 가중치와 데이터셋은 [링크](https://1drv.ms/u/s!AuS4E8bOdN2Blpd1jw9ilzNhT0dI3Q?e=yKGlel)에서 다운로드 (기한 2020년 6월 20일)
* Goal
  * Adversarial attack 에 강건한 모델 만들기



In [ ]:
import os
import numpy as np
import pickle
import tensorflow as tf
from tensorflow.keras import models, layers

In [ ]:
print('tf',tf.__version__)

## 데이터 불러오기
* 데이터는 앞서 다운로드 링크 제공

In [ ]:
X_test = np.load('../data/X_test.npy')
y_test = np.load('../data/y_test.npy')

print(X_test.shape, y_test.shape)

## 사전학습 된 모델 불러오기
* 모델 구조는 아래 Model 클래스와 같음
* 사전학습 된 가중치는 앞서 다운로드 링크 제공
* 모델 학습에 관한 상세한 과정은 <span style="color:red;background-color:pink">1. how to pretrain the model.ipynb</span> 파일을 참고 바랍니다.

In [ ]:
class Model(models.Model):
    def __init__(self,base_model):
        super(Model,self).__init__()
        self.base_model = base_model
        self.top_layer = models.Sequential([
            layers.Dense(10),
            layers.Activation(tf.nn.softmax),
        ])
        
    def call(self,inputs,training=False):
        x = self.base_model(inputs, training=training)
        x = layers.Flatten()(x)
        outputs = self.top_layer(x, training=training)
        return outputs

In [ ]:
# weight loading
with open('../model/model_20200507_9_1.00_0.0088','rb') as w:
    weights = pickle.load(w)

base_model = tf.keras.applications.MobileNetV2(input_shape=[160, 160, 3], include_top=False,weights='imagenet')

# white-box model
model = Model(base_model)
model.build((None,160,160,3))
model.set_weights(weights)